In [60]:
import numpy as np
import pandas as pd
from bokeh.layouts import row, widgetbox, column
from bokeh.models import CustomJS, Slider, Select, HoverTool 
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import push_notebook, output_notebook, curdoc
from bokeh.client import push_session
from bokeh.charts import Scatter
#from bokeh.scatter_with_hover import scatter_with_hover
output_notebook()

Loading BokehJS ...

In [16]:
np.random.seed(0)
samples = np.random.randint(low = 0, high = 1000, size =  1000)
samples = samples.reshape(200,5)
cols = ["A", "B", "C", "D", "E"]
df = pd.DataFrame(samples, columns=cols)

# Here is a dict of some keys that I want to be able to pick from for plotting
labels = list(df.columns.values)
axis_map = {key:key for key in labels}

df.tail()

,A,B,C,D,E
195,920,551,234,635,284
196,664,658,707,172,723
197,301,822,0,138,707
198,902,731,867,441,966
199,915,162,50,242,870


In [54]:
#This is to update during the callback
#original example in working minimal example


code2 = ''' var data = source.data;
            
           //axis values with select widgets
           var value1 = val1.value;
           var value2 = val2.value;
                     
           var original_data = original_source.data
           // get data corresponding to selection
           
           x = original_data[value1];
           y = original_data[value2];
           //data['x'] = x;
           //data['y'] = y;
           
           var start1 = Math.min.apply(Math, original_data[value1])
           var end1 = Math.max.apply(Math, original_data[value1])
           
           var start2 = Math.min.apply(Math, original_data[value2])
           var end2 = Math.max.apply(Math, original_data[value2])
           

           
           //change slider values
           slide.set("start", start1)
           slide.set("end", end1)
           slide.set("step", (end1 -start1)/20.)
           //slide.set("title", val1)
                      
           slide2.set("start", start2)
           slide2.set("end", end2)
           slide2.set("step", (end2 -start2)/20.)
           //slide2.set("title", val2)
           
           slide3.set("start", start1)
           slide3.set("end", end1)
           slide3.set("step", (end1 -start1)/20.)
           //slide3.set("title", val1+"end")
           
           slide4.set("start", start2)
           slide4.set("end", end2)
           slide4.set("step", (end2 -start2)/20.)
           //slide4.set("title", val2+"end")          
          
           
           
           //source.trigger('change');
           
           
           // set axis labels
           x_axis.axis_label = value1;
           y_axis.axis_label = value2;
           
           // #need to create filter based on min and max vals

           
           var val1 = slide.value
           var val2 = slide2.value
           var val3 = slide3.value
           var val4 = slide4.value
           
           function inBetween2X(x) {
               return (val1 < x & x < val3)

           }
           
           function inBetween2Y(x) {
               return (val2 < x & x<val4)
       
           }
           
           source.trigger('change');
           filter_x = x.filter(inBetween2X);
           filter_y = y.filter(inBetween2Y);
                  
           data['x'] = filter_x;
           data['y'] = filter_y;
                  
           console.log(filter_y.length)
           console.log(filter_x.length)
            
           console.log("Code Syntax is atleast correct")
           
           '''


#cols = ["beta12", "beta23"]

source = ColumnDataSource(data=dict( x=df['A'], y=df['B'], 
                                    label = labels))
original_source = ColumnDataSource(data=df.to_dict(orient='list'))

#TOOLS = [ HoverTool(tooltips= [(c, '@' + c) for c in df.columns] + [('index', '$index')])]
TOOLS = [HoverTool(tooltips = 
                  [("A", "@A"),
                   ("Index", "$index")
                      
                  ])]
# hover.tooltips.append(('index', '$index'))

#plot the figures
plot = figure(plot_width=800, plot_height=800,   tools= TOOLS)
plot.scatter(x= "x",y="y", source=source, line_width=2, line_alpha=0.6, 
             size = 3)

slider = Slider(title ="Slider X Start", start=df['A'].min(), end=df['A'].max(),
                step=(df['A'].max() - df['A'].min()) /10., 
               value = df['A'].min())
slider2 = Slider(title ="Slider Y Start", start=df['B'].min(), end=df['B'].max(),
                 step=(df['B'].max() - df['B'].min())/10., 
                 value = df['B'].min())


slider3 = Slider(title ="Slider X End", start=df['A'].min(), end=df['A'].max(),
                step=(df['A'].max() - df['A'].min()) /10., 
               value = df['A'].max())
slider4 = Slider(title ="Slider Y ENd", start=df["B"].min(), end=df['B'].max(),
                 step=(df['B'].max() - df['B'].min())/10., 
                 value = df['B'].max())


callback = CustomJS(args=dict(source=source, original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code2)




#create a separate callback to manage chaning data
callbackDRange = CustomJS(args = dict(source = source,original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code2 )

slider.callback = callback
callback.args["slide"] = slider
#callbackDRange.args["slide"] = slider


slider2.callback = callback
callback.args["slide2"] = slider2
#callbackDRange.args["slide2"] = slider2

slider3.callback = callback
callback.args["slide3"] = slider3
#callbackDRange.args["slide3"] = slider3

slider4.callback = callback
callback.args["slide4"] = slider4
#callbackDRange.args["slide4"] = slider4


#Create two select widgets to pick the features of interest 
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="A", callback = callback)
callback.args["val1"] = x_axis
callbackDRange.args["val1"]= x_axis

y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="B", callback = callback)
callback.args["val2"] = y_axis
callbackDRange.args["val2"]= y_axis

y_axis.js_on_change("value", callback)

x_axis.js_on_change("value", callback)


plot.xaxis[0].axis_label = 'A'
plot.yaxis[0].axis_label = 'B'
#Error message due to legend, need to find a better way to do so

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


In [55]:
#Display the graph in a jupyter notebook
layout = column(plot, x_axis, y_axis, row(slider, slider2),row(slider3, slider4) )
show(layout, notebook_handle=True)

In [83]:
code2 = ''' var data = source.data;
            
           //axis values with select widgets
           var value1 = val1.value;
           var value2 = val2.value;
                     
           var original_data = original_source.data
           // get data corresponding to selection
           
           x = original_data[value1];
           y = original_data[value2];
           data['x'] = x;
           data['y'] = y;
           
           source.trigger('change');
                      
           // set axis labels
           x_axis.axis_label = value1;
           y_axis.axis_label = value2;

           '''
#datas = "datas"
source = ColumnDataSource(data=dict( x=df['A'], y=df['B'], 
                                    label = labels, data_vals = df))
#data=dict( x=df['A'], y=df['B'], label = labels, datas = df)

original_source = ColumnDataSource(data=df)#.to_dict(orient='list'))

#a=  source.data[datas].columns.values
#print a.columns.values
#print a

TOOLS = [ HoverTool(tooltips= [(c, '@' + c) for c in  source.data["data_vals"].columns.values] +
                               [('index', '$index')] )]


#TOOLS = [HoverTool(tooltips = [ ("A", "@A"  )])]  

# hover.tooltips.append(('index', '$index'))
#plot the figures
plot = figure(plot_width=800, plot_height=800,   tools= TOOLS)
plot.scatter(x= "x",y="y", source=source, line_width=2, line_alpha=0.6, 
             size = 3)

callback = CustomJS(args=dict(source=source, original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0]), code=code2)

#Create two select widgets to pick the features of interest 
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="A", callback = callback)
callback.args["val1"] = x_axis

y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="B", callback = callback)
callback.args["val2"] = y_axis



plot.xaxis[0].axis_label = 'A'
plot.yaxis[0].axis_label = 'B'
#Error message due to legend, need to find a better way to do so


#Display the graph in a jupyter notebook
layout = column(plot, x_axis, y_axis )
show(layout, notebook_handle=True)

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


In [82]:
vals = source.data["data_vals"]
print vals
#r = scatter.renderers[0]
#pd.DataFrame(r.data_source.data)

       A    B    C    D    E
0    684  559  629  192  835
1    763  707  359    9  723
2    277  754  804  599   70
3    472  600  396  314  705
4    486  551   87  174  600
5    849  677  537  845   72
6    777  916  115  976  755
7    709  847  431  448  850
8     99  984  177  755  797
9    659  147  910  423  288
10   961  265  697  639  544
11   543  714  244  151  675
12   510  459  882  183   28
13   802  128  128  932   53
14   901  550  488  756  273
15   335  388  617   42  442
16   543  888  257  321  999
17   937   57  291  870  119
18   779  430   82   91  896
19   398  611  565  908  633
20   938   84  203  324  774
21   964   47  639  131  972
22   868  180  846  143  660
23   227  954  791  719  909
24   373  853  560  305  581
25   169  675  448   95  197
26   606  256  881  690  292
27   930  816  861  387  610
28   554  973  368  999  917
29   201  383  512  906  370
..   ...  ...  ...  ...  ...
170  754  313  475  345  387
171  125  718  850  197  698
172  900   17 